**Notebook to train several models with different training/data configurations**

The notebook is designed to run on colab. It can be easily adapted to run locally, with a few imports.

In [ ]:
# This is the path to the helper functions folder
# On drive for example: '/content/drive/My Drive/myfolder/helper_functions'
modules_path = './helper_functions'
models_path = './models' # for the models folder

# This is the path to the data folder
# On drive for example: '/content/drive/myfolder/data'
base_path = './data/'

# This is the path to the tensorflow checkpoint folder
# On drive for example: '/content/drive/MyDrive/best_models'
checkpoint_filepath = './best_models'

In [ ]:
# pip calls, COLAB STUFF
!pip install tensorflow_addons

# all the nice imports <3
import os
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import h5py
import numpy as np
from datetime import datetime
import keras
import importlib
import pickle

# black magic, so that tf.Tensor objects can be used as numpy things
# all hail tensorflow 0_0
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# show important library versions
print("TensorFlow version: {}".format(tf.__version__))
print("TensorFlow Datasets version: ",tfds.__version__)


# add custom paths (to import the nice models and helper classes)
import sys
sys.path.append(models_path)
sys.path.append(modules_path)

# add custom imports here
import processing
import metrics
import conv_architectures as ca
import autoencoder_models as am

# Very Google Colab specific stuff

In [ ]:
# import & mount drive
from google.colab import drive
drive.mount('/content/drive')

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  print('Warning: TPU device not found')

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('Warning: GPU device not found')
else:  
  print('Found GPU at: {}'.format(device_name))


# Tensorboard stuff (callbacks for logging data)
%load_ext tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Set up of runs
**Walkthorugh**
1. Give the problem a name for file naming and create the PROBLEM FOLDER, along with a sub-folder called "history" 
2. The data sets are named based on sample size (size) and data set size (dataset_size). These can be specified below
3. All parameters that can be varied (or that there exists data sets for)
4. Custom definitions, and defintions of the models (dictionary with: model class, designation, and regularization)

In [ ]:
# 1.
PROBLEM = 'DS'
PROBLEM_FOLDER = 'my_experiments_folder'

# 2.
dataset_size = '10000'
size = '32' # or 32 or 64

size_path = size + 'x' + size + '/'
model_base_path = './'+ PROBLEM_FOLDER + '/' + size_path


# 3.
cr_array = np.array(['0.1000', '0.0500', '0.0200', '0.0100', '0.0050', '0.0020']) # contrast_ratios
ds_size_array = np.array([1000,2000,5000,10000]) # data set sizes
twopoint_array = np.array([False, True]) # using two-point correlations

# 4.
model_array = np.array([
    {'model' : ca.Conv32Model1, 'name' : 'large', 'reg': 0.001},
    # {'model' : ca.ConvModel3, 'name' : 'medium', 'reg': 0.005},
    {'model' : ca.Conv32Model3, 'name' : 'small', 'reg': 0.01},
    ])

cr_array = np.array(['0.0100'])
ds_size_array = np.array([1000, 2000, 5000])
twopoint_array = np.array([False])
EPOCHS = 1000
REG = 0.001



# Execute Runs

In [ ]:
results_array = []
FILENAME = 'all_data_' + PROBLEM
TEST_SIZE = 1000
SPLITS = np.array([0.9,0,0.1])

counter = 0

for model_data in model_array:
  for cr in cr_array:
    for ds in ds_size_array:
      for tp in twopoint_array:

        counter += 1
        print("Currently working on model No.", counter)

        data_path = base_path + size_path + 'n=' + size + '_x=' + dataset_size + '_cr=' + cr + '_DS.h5'
        tp_data_path = base_path + size_path + 'n=' + size + '_x=' + dataset_size + '_cr=' + cr + '_TP.h5'

        f = h5py.File(data_path, 'r')
        imgs = np.array(f['images'])
        labs = np.array(f['c_vectors'])

        print(labs[0])

        twopoint_f = h5py.File(tp_data_path, 'r')
        tp_imgs = np.array(twopoint_f['twopoint'])

        f.close()
        twopoint_f.close()

        if tp:
          ims = tp_imgs


        dataproc = processing.Processing(
          imgs[:ds], 
          labs[:ds], 
          SPLITS, 
          custom_valid = (imgs[-1000:-500], labs[-1000:-500]),
          custom_test = (imgs[-500:], labs[-500:]),
          batch_size=256, 
          conv_behavior=True, 
          scale_labels=False, 
          symmetric=True, 
          twopoint=False,
          shuffle=False,
          pca=False,
          # pca_variance=0.7
          )
        
        model = model_data['model'](dataproc, regularization=model_data['reg'])
        print(model.summary())

        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_r_square',
            mode='max',
            save_best_only=True)


        params = {
            'x' : dataproc.ds_train,
            'batch_size' : dataproc.batch_size, 
            'epochs' : EPOCHS, 
            'validation_data' : dataproc.ds_val, 
            'verbose' : 1, 
            'callbacks' : [model_checkpoint_callback]
            # 'callbacks' : [tensorboard_callback, model_checkpoint_callback]
        }

        model.compile(optimizer="Adam", loss="mse", metrics=[tfa.metrics.r_square.RSquare()])
        history = model.fit(**params)

        model.load_weights(checkpoint_filepath)

        test_metrics = model.evaluate(dataproc.ds_test)

        custom_path = PROBLEM + '_model' + model_data['name'] + '_ds' + str(dataproc.splits[0]) + '_tp' + str(tp) + '_epochs' + str(params['epochs']) + '_cr' + cr
        total_path = model_base_path + custom_path
        model.save(total_path)

        results_array.append(
            {
                'run_name' : custom_path,
                'model_name' : model_data['name'],
                'regularization' : model_data['reg'],
                'ds_size' : dataproc.splits[0],
                'twopoint' : tp,
                'contrast_ratio' : cr,
                'epochs' : params['epochs'],
                'history' : history.history,
                'test_metrics' : test_metrics,
                'model_function' : model_data['model'],
                'path_to_weights' : total_path,
            }
        )


with open(model_base_path + 'history/' + FILENAME, 'wb') as file_pi:
  pickle.dump(results_array, file_pi)


In [ ]:
%tensorboard --logdir logs/scalars

# Examples and use cases

In [ ]:
# EXAMPLE: Reopen files

mymod = ca.ConvModel2(dataproc)
print(mymod.summary())

mymod.load_weights(model_base_path + 'modelmedium_ds4250_tpTrue_epochs5')

mymod.compile(optimizer="Adam", loss="mse", metrics='accuracy')
mymod.evaluate(dataproc.ds_test)

In [ ]:
# EXAMPLE: Plotting data

with open(PROBLEM_FOLDER + 'history/all_data_cr', 'rb') as f:
    results = pickle.load(f)

length = len(results[0]['history']['val_r_square'])
epochs = np.linspace(1,length, num=length)

for res in results:
  # print(res['history'])
  mn = res['model_name'].upper()[0] + str(res['twopoint']) + "_cr" + res['contrast_ratio'] + '_ds' + str(res['ds_size'])
  y = res['history']['val_r_square']
  if res['model_name'] == 'small' or True:
    print(mn, "Average value", np.mean(y), "max val", np.max(y))
    plt.plot(epochs,y, label=mn)

plt.legend(loc='lower right')
ax = plt.gca()
ax.set_xlim([0, length])
ax.set_ylim([0.3, 1])

plt.show()
